# KoNLpy

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,374 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 1,710 kB in 1s (1,177 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install python-mecab-ko

mecab-ko is already installed
mecab-ko-dic is already installed
Install mecab-python
/tmp /content
/content
Processing /tmp/mecab-python-0.996
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
# 형태소 분석기 4개 import
from konlpy.tag import Okt
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from PIL import Image
import numpy as np
import pandas as pd

import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from mecab import MeCab

In [ ]:
Okt = Okt()
hannanum = Hannanum()
komoran = Komoran()
kkma = Kkma()

mecab = MeCab()

# 실제 전처리 시작

## 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_colwidth', 1000)

In [ ]:
df = '본인의 데이터 파일 경로'

In [ ]:
df.info()

In [ ]:
# 결측치 확인
df['내용'][df['내용'].isnull() == True]

In [ ]:
# 결측치 대체
null_list = [rows 확인해서 써주시길 ..]
for i in null_list:
    df.loc[i,'내용'] = "내용없음"

### Hannanum

In [ ]:
# 정규화
def preprocess(text):
    text = str(text)
    text = text.strip() # 텍스트 시작과 끝에 있는 공백 제거
    text = text.replace("文","문재인") # 한자 대체
    text = re.sub(r'\[.*?\]', '', text) # []와 그 안에 있는 내용 제거
    text = re.sub(r'\S+@\S+',' ',text)  # 이메일 주소 제거
    text = re.sub(r'\S+\s?뉴스',' ',text) # 뉴스 이름 제거
    text = re.sub(r'\S+\s?기자',' ',text) # 기자 이름 제거
    text = re.sub('\s+', ' ', text)  # 여러 개의 연속 공백을 하나의 공백으로 대체
    text = re.sub(r'[^\w\s]', ' ', str(text).strip()) # 단어, 공백 아닌 것들 띄어쓰기로 대체
    #text = re.sub(r'\d',' ',text) # 숫자 제거
    return text

# 한글자 제외, 불용어 제거
def extract_hannanum(text):
    text_split = hannanum.pos(text,ntags=22)
    text_list=[]
    for word in text_split:
      if len(word[0])>1 and word[1] not in ['JC','JX','JP','EP','EF','EC','ET','XP','XS','S']: #Adverb, Adjective
        text_list.append(word[0])
    return text_list

# 최종
def finalpreprocess_hannanum(text):
  return extract_hannanum(preprocess(text))

In [ ]:
tqdm.pandas(desc="Processing")
df['내용 전처리'] = df['내용'].progress_apply(preprocess)

Processing: 100%|██████████| 18927/18927 [00:11<00:00, 1600.34it/s]


In [ ]:
tqdm.pandas(desc="Preprocessing")
df['내용 2차 전처리'] = df['내용'].progress_apply(finalpreprocess_hannanum)

Preprocessing: 100%|██████████| 18927/18927 [1:03:41<00:00,  4.95it/s]


In [ ]:
def keyword_extraction(text):
    keywords = re.findall(r"‘(.*?)’", str(text))
    return keywords

In [ ]:
df['키워드'] = df['내용'].apply(keyword_extraction)

In [ ]:
df

,Unnamed: 0,작성일,작성자,제목,내용,내용 전처리,내용 2차 전처리,키워드
0,0,2019/05/01,파이낸셜뉴스,박승원 광명시장 “광명형 청년정책 본격 추진”,"광명시‘시장 직속 청년 위원회’ 출범식. 사진제공=광명시 [광명=파이낸셜뉴스 강근주 기자] 광명형 청년정책을 만들어내는 요람이 탄생했다. 광명시는 30일 시청 대회의실에서 청년위원 50명이 참석한 가운데 ‘시장 직속 청년위원회’ 출범식을 개최했다. 박승원 광명시장은 이날 출범식에서 “시장직속 청년위원회를 중심으로 분야별 청년사업에 대해 민-관이 함께 협업하고 심도 있게 논의해, 청년에게 실질적으로 필요한 광명형 청년정책을 만들어 나가겠다”고 밝혔다. 청년위원회는 3월26일 제정된 ‘광명시 청년 기본조례’에 따라 청년의 권리 보호 및 신장, 복지 향상, 청년 삶의 질 개선을 위해 분야별 청년정책, 신규사업 발굴 등 자문·심의 기능을 하게 된다. 청년 민간전문가, 교수, 부서장 등 민관이 함께 청년위원회에 참여하고, 전체 위원의 절반 이상이 다양한 청년(다문화 및 장애인 청년, 대학생, 청년인턴, 청년직장인, 청년CEO, 청년문화예술가, 청년노무사, 청년네트워크 활동가 등)으로 구성됐다. 청년위원회는 청년참여-청년지원-청년안정 3개 분과로 나뉘어 청년기본계획 수립을 비롯해 청년생각펼침 공모사업, 청년욕구조사, 청년센터 설립, 청년주택, 광명형 청년정책 등을 논의한다. 한편 광명시는 이날 청년정책 기본계획 착수보고회를 함께 개최했으며, 2019년부터 2023년까지 단계별 및 중장기 계획을 마련하고, 중앙부처 및 타 지역 사례를 통해 청년정책을 제언할 계획이다. kkjoo0912@fnnews.com 강근주 기자 ※",광명시 시장 직속 청년 위원회 출범식 사진제공 광명시 광명형 청년정책을 만들어내는 요람이 탄생했다 광명시는 30일 시청 대회의실에서 청년위원 50명이 참석한 가운데 시장 직속 청년위원회 출범식을 개최했다 박승원 광명시장은 이날 출범식에서 시장직속 청년위원회를 중심으로 분야별 청년사업에 대해 민 관이 함께 협업하고 심도 있게 논의해 청년에게 실질적으로 필요한 광명형 청년정책을 만들어 나가겠다 고 밝혔다 청년위원회는 3월26일 제정된 광명시 청년 기본조례 에 따라 청년의 권리 보호 및 신장 복지 향상 청년 삶의 질 개선을 위해 분야별 청년정책 신규사업 발굴 등 자문 심의 기능을 하게 된다 청년 민간전문가 교수 부서장 등 민관이 함께 청년위원회에 참여하고 전체 위원의 절반 이상이 다양한 청년 다문화 및 장애인 청년 대학생 청년인턴 청년직장인 청년CEO 청년문화예술가 청년노무사 청년네트워크 활동가 등 으로 구성됐다 청년위원회는 청년참여 청년지원 청년안정 3개 분과로 나뉘어 청년기본계획 수립을 비롯해 청년생각펼침 공모사업 청년욕구조사 청년센터 설립 청년주택 광명형 청년정책 등을 논의한다 한편 광명시는 이날 청년정책 기본계획 착수보고회를 함께 개최했으며 2019년부터 2023년까지 단계별 및 중장기 계획을 마련하고 중앙부처 및 타 지역 사례를 통해 청년정책을 제언할 계획이다,"[광명시, 시장, 직속, 청년, 위원회, 출범식, 사진제공, 광명시, 광명형, 청년정책, 만들, 요람, 탄생, 광명시, 30, 시청, 대회의실, 청년위원, 50, 참석, 가운데, 시장, 직속, 청년위원회, 출범식, 개최, 박승원, 광명시장, 출범식, 시장직속, 청년위원회, 중심, 분야별, 청년사업, 함께, 협업, 논의해, 청년, 실질적, 필요, 광명형, 청년정책, 만들, 밝히, 청년위원회, 3월26, 제정, 광명시, 청년, 기본조례, 따르, 청년, 권리, 보호, 신장, 복지, 향상, 청년, 개선, 위하, 분야별, 청년정책, 신규사업, 발굴, 자문, 기능, 청년, 민간전문가, 교수, 부서장, 민관, 함께, 청년위원회, 참여, 전체, 위원, 절반, 이상, 다양, 청년, 다문화, 장애, 청년, 대학생, 청년인턴, 청년직장, 청년CEO, 청년문화예술, 청년노무사, 청년네트워크, 활동, 구성, 청년위원회, 청년참여, 청년지원, 청년안정, 분과, 나뉘, 청년기본계획, 수립, ...]","[시장 직속 청년 위원회, 시장 직속 청년위원회, 광명시 청년 기본조례]"
1,1,2019/05/01,강원일보,[강릉]청년정책보고회 개최,"【강릉】강릉시는 지난 30일 시청 18층 전망대에서 현재 청년 관련 정책을 추진하고 있는 10개 부서 부서장 및 담당 등 20여명이 참석한 가운데 `청청토크 (靑靑TALK )강릉시 청년정책 보고회'를 열고 정책 추진을 위한 내부적 공감 확산에 나섰다. 현재 일자리경제과는 통합 일자리 플랫폼 구축, 강원도 사회보험료 지원, 도 일자리 안심공제 지원, 청년 상인 야시장, 일자리 사업 등 11개 사업을 추진 중이며 기업지원과는 사회맞춤형 산학협력 선도대학 육성지원, 창업보육센터 등의 사업을 펼치고 있다. 조상원기자",강릉 강릉시는 지난 30일 시청 18층 전망대에서 현재 청년 관련 정책을 추진하고 있는 10개 부서 부서장 및 담당 등 20여명이 참석한 가운데 청청토크 靑靑TALK 강릉시 청년정책 보고회 를 열고 정책 추진을 위한 내부적 공감 확산에 나섰다 현재 일자리경제과는 통합 일자리 플랫폼 구축 강원도 사회보험료 지원 도 일자리 안심공제 지원 청년 상인 야시장 일자리 사업 등 11개 사업을 추진 중이며 기업지원과는 사회맞춤형 산학협력 선도대학 육성지원 창업보육센터 등의 사업을 펼치고 있다,"[강릉, 강릉시, 지나, 30, 시청, 18, 전망대, 현재, 청년, 관련, 정책, 추진, 10, 부서장, 담당, 20, 참석, 가운데, 청청토크, 靑靑TALK靑靑TALK, 강릉시, 청년정책, 보고회, 정책, 추진, 위하, 내부적, 공감, 확산, 나서, 현재, 일자리경제, 통합, 일자리, 플랫폼, 구축, 강원, 사회보험료, 지원, 일자리, 안심공제, 지원, 청년, 야시장, 일자리, 사업, 11, 사업, 추진, 기업지원, 사회맞춤형, 산학협력, 선도대학, 육성지원, 창업보육센터, 사업, 펼치]",[]
2,2,2019/05/01,뉴스1,"文대통령, 사회원로와 오찬 간담회…2일 주요일정","문재인 대통령. (청와대 제공) 2019.4.30/뉴스1 ◇文대통령, 김영란·윤여준 등 사회원로 12명과 오찬 간담회 문재인 대통령은 2일 사회원로 12명을 청와대로 초청, 오찬간담회를 갖고 최근 남북관계·여야 대치국면·추가경정예산안 등 산적한 국정현안에 대한 고견을 구한다. 이날 간담회는 진보와 보수를 넘어 사회통합적 측면에서 원로들을 초청한 점이 눈에 띤다. 김영란 전 대법관과 조한혜정 연세대 명예교수는 진보적 인사로, 김영삼 정부 때 환경부 장관을 지낸 윤여준 전 장관은 보수 성향으로 분류된다. 이밖에 김명자 한국과학기술단체총연합회 회장, 김우식 국민안전안심위원회 위원장, 김지형 전 신고리 5·6호기 공론화위원회 위원장이 참석한다. 또 송호근 포항공대 석좌교수, 안병욱 한국학중앙연구원 원장, 이종찬 전 국가정보원장, 이홍구 전 국무총리, 정해구 대통령 직속 정책기획위원회 위원장, 조은 동국대 명예교수 등이 함께 한다. 지난해 5월4일 필리핀 마닐라 샹그릴라호텔에서 열린 한중일 재무장관·중앙은행 총재회의에 앞서 이주열 한국은행 총재(왼쪽)가 구로다 하루히코(黑田東彦.중앙) 일본은행 총재 , 아소다로(麻生太郞) 일본 부총리 겸 재무상과 이야기를 나누고 있다. (한국은행 제공) © News1 장도민 기자 ◆ 한중일 재무장관·중앙은행 총재 회의...역내 금융안전망 강화 등 논의 2일(현지시간) 오전 피지에서 한중일 재무장관·중앙은행 총재 회의가 열린다. 홍남기 경제부총리 겸 기획재정부 장관과 이주열 한국은행 총재는 이날 회의에서 한국 경제 동향 및 정책을 소개하고 한중일 3국의 공동 협력방안을 논의한다. 아세안(ASEAN)+3(한중일) 역내 금융안전망인 치앙마이 이니셔티브(CIMM) 지원 체계를 강화하는 협정문 등이 주요 주제다. 회의 결과는 한국시간으로 오전 9시에 발표된다. 한중일 재무장관·중앙은행 총재 회의는 매년 ADB(아시아개발은행) 총회 및 아세안+3 재무장관·중앙은행 총재 회의 때 개최되고 있다. 홍영표 더불어민주당 원내대표 등 참석...",문재인 대통령 청와대 제공 1 문재인대통령 김영란 윤여준 등 사

In [ ]:
df.to_csv('저장경로/파일명.csv',index=False)

## 리스트 구문으로 처리(위에 잘 됐으면 무시해도됨)

In [ ]:
cheongnyeon_news_list = cheongnyeon_news['내용'].tolist()

In [ ]:
for i in range(len(cheongnyeon_news_list)):
    print('--------------')
    print(i+1, '/', len(cheongnyeon_news_list))
    text_split = []
    #error_list = []
    text = cheongnyeon_news_list[i]
    text_split.append(finalpreprocess_hannanum(text))
    #print(finalpreprocess_hannanum(text))
print('The end')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
16428 / 18927
--------------
16429 / 18927
--------------
16430 / 18927
--------------
16431 / 18927
--------------
16432 / 18927
--------------
16433 / 18927
--------------
16434 / 18927
--------------
16435 / 18927
--------------
16436 / 18927
--------------
16437 / 18927
--------------
16438 / 18927
--------------
16439 / 18927
--------------
16440 / 18927
--------------
16441 / 18927
--------------
16442 / 18927
--------------
16443 / 18927
--------------
16444 / 18927
--------------
16445 / 18927
--------------
16446 / 18927
--------------
16447 / 18927
--------------
16448 / 18927
--------------
16449 / 18927
--------------
16450 / 18927
--------------
16451 / 18927
--------------
16452 / 18927
--------------
16453 / 18927
--------------
16454 / 18927
--------------
16455 / 18927
--------------
16456 / 18927
--------------
16457 / 18927
--------------
16458 / 18927
--------------
16459 / 18927
--------------
16460 / 18927
--------------
16461 /

In [ ]:
text_split = []

In [ ]:
cheongnyeon_news['내용 전처리'] = text_split

In [ ]:
cheongnyeon_news.head()